In [1]:
!uv add beautifulsoup4
!uv add result
!uv add pandas

Resolved 110 packages in 0.32ms
Audited 106 packages in 0.05ms
Resolved 110 packages in 0.33ms
Audited 106 packages in 0.05ms


In [15]:
import requests
from result import Ok, Err, Result, is_ok, is_err
import duckdb
from pathlib import Path


def create_duck_db_table():
    duckdb.sql("""CREATE OR REPLACE TABLE

    thai_population
    (
        data_year INTEGER,
        yymm VARCHAR,
        cc_code INTEGER,
        cc_desc VARCHAR,
        rcode_code VARCHAR,
        rcode_desc VARCHAR,
        ccaatt_code VARCHAR,
        ccaatt_desc VARCHAR,
        ccaattmm_code VARCHAR,
        ccaattmm_desc VARCHAR,
        male INTEGER,
        female INTEGER,
        total INTEGER,
        house INTEGER,
        PRIMARY KEY (data_year, cc_code)
    );""")
    
def generate_insert_sql(
    data_year: int,
    yymm: str,
    cc_code: int,
    cc_desc: str,
    rcode_code: str,
    rcode_desc: str,
    ccaatt_code: str,
    ccaatt_desc: str,
    ccaattmm_code: str,
    ccaattmm_desc: str,
    male: int,
    female: int,
    total: int,
    house: int,
) -> str:

    return f"""INSERT INTO thai_population
VALUES (
    '{data_year}',
    '{yymm}',
    '{cc_code}',
    '{cc_desc}',
    '{rcode_code}',
    '{rcode_desc}',
    '{ccaatt_code}',
    '{ccaatt_desc}',
    '{ccaattmm_code}',
    '{ccaattmm_desc}',
    '{male}',
    '{female}',
    '{total}',
    '{house}'
)"""

def write_into_hive_partition():
    

    Path(".datasets").mkdir(parents=True, exist_ok=True)
    duckdb.sql("""COPY thai_population
TO
    './datasets/thai_population'
    (
        FORMAT PARQUET,
        PARTITION_BY (data_year),
        OVERWRITE_OR_IGNORE,
        COMPRESSION GZIP,
        FILE_EXTENSION 'parquet.gz'
    )
""")
write_into_hive_partition()

def convert_to_thai_year(year: int) -> int:
    """Convert year into shortened Thai year

    Args:
        year (int) - Year such as 2024
    Returns:
        int - (Thai year)
    """
    return (year + 543) - 2500

def get_data_stat_by_year(year: int) -> Result[str, str]:
    thai_year = convert_to_thai_year(year) 
    url = f"https://stat.bora.dopa.go.th/new_stat/file/{thai_year}/stat_c{thai_year}.txt"
    response = requests.get(url)
    if not response.ok:
        return Err(response.text)
    else:
        return Ok(response.text)


def clean_text(text: str) -> str:
    return text.strip("\ufeff")


def extract_row(row: str) -> list[str]:
    return row.split("|")

def string_to_int(value: str) -> int:
    return int(value.replace(",", ""))



year = 1993


# Create table
create_duck_db_table()
while True:
    

    # Match pattern
    match get_data_stat_by_year(year):
        case Ok(value):
            ...
        case  Err(e):
            break
    # Create a data structure
    table = clean_text(value)
    
    # Insert into the table
    for row in table.splitlines():
        (
            yymm,
            cc_code,
            cc_desc,
            rcode_code,
            rcode_desc,
            ccaatt_code,
            ccaatt_desc,
            ccaattmm_code,
            ccaattmm_desc,
            male_str,
            female_str,
            total_str,
            house_str,
        ) = extract_row(row.strip("|"))
        sql_syntax = generate_insert_sql(
            year,
            yymm,
            string_to_int(cc_code),
            cc_desc,
            rcode_code,
            rcode_desc,
            ccaatt_code,
            ccaatt_desc,
            ccaattmm_code,
            ccaattmm_desc,
            string_to_int(male_str),
            string_to_int(female_str),
            string_to_int(total_str),
            string_to_int(house_str),
        )
        duckdb.sql(sql_syntax)
    year += 1

In [36]:
duckdb.sql("""SELECT *
FROM read_parquet('./datasets/thai_population/*/*.parquet.gz', hive_partitioning = true);
""").to_df()

,yymm,cc_code,cc_desc,rcode_code,rcode_desc,ccaatt_code,ccaatt_desc,ccaattmm_code,ccaattmm_desc,male,female,total,house,data_year
0,3612,0,ทั่วประเทศ,0,,0,,0,,29205086,29130986,58336072,13336167,1993
1,3612,10,กรุงเทพมหานคร,0,,0,,0,,2760480,2812232,5572712,1472621,1993
2,3612,11,จังหวัดสมุทรปราการ,0,,0,,0,,442043,453341,895384,267935,1993
3,3612,12,จังหวัดนนทบุรี,0,,0,,0,,352531,364874,717405,224651,1993
4,3612,13,จังหวัดปทุมธานี,0,,0,,0,,247964,252122,500086,160164,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,6612,92,จังหวัดตรัง,0,,0,,0,,311212,325805,637017,243956,2023
2396,6612,93,จังหวัดพัทลุง,0,,0,,0,,253232,267366,520598,205036,2023
2397,6612,94,จังหวัดปัตตานี,0,,0,,0,,364396,372681,737077,204902,2023
2398,6612,95,จังหวัดยะลา,0,,0,,0,,273248,276698,549946,178717,2023


Resolved 114 packages in 1.20s                                       
⠙ Preparing packages... (0/4)                                                   ⠋ Resolving dependencies...                                                     
⠙ Preparing packages... (0/4)----     0 B/338.46 KiB                    
tzdata     ------------------------------     0 B/338.46 KiB
⠙ Preparing packages... (0/4)----     0 B/496.10 KiB                    
tzdata     ------------------------------ 16.00 KiB/338.46 KiB
⠙ Preparing packages... (0/4)----     0 B/496.10 KiB                    
tzdata     ------------------------------ 32.00 KiB/338.46 KiB
⠙ Preparing packages... (0/4)----     0 B/496.10 KiB                    
tzdata     ------------------------------ 48.00 KiB/338.46 KiB
⠙ Preparing packages... (0/4)----     0 B/496.10 KiB                    
tzdata     ------------------------------ 63.09 KiB/338.46 KiB
⠙ Preparing packages... (0/4)----     0 B/496.10 KiB                    
tzdata     -------

⠸ Preparing packages... (2/4)---- 3.76 MiB/10.80 MiB                    
⠸ Preparing packages... (2/4)---- 5.58 MiB/10.80 MiB                    
Prepared 4 packages in 563ms                                                 
Installed 4 packages in 48ms                                
 + numpy==2.2.0
 + pandas==2.2.3
 + pytz==2024.2
 + tzdata==2024.2


In [42]:
x1 = "6212|95|จังหวัดยะลา |0| |0| |0| |266,860|269,470|536,330|167,137|".strip("|").split("|")
len(x1)

13

In [40]:
x2 = "6312|0|ทั่วประเทศ|0| |0| |0| |32,375,532|33,811,195|66,186,727|27,224,743|".strip("|").split("|")
len(x2)

13